In [1]:
from ipywidgets import FloatProgress

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('/work/seongjinpark/tomcat-speech/speech_recognition'))

In [3]:
from helper.preprocessing import generate_dict
from helper.wav2vec_helper import load_train_test

In [4]:
from ipywidgets import FloatProgress
from datasets import load_dataset, load_metric

# train_path = "/data/seongjinpark/chalearn/train/audio_16"
# test_path = "/data/seongjinpark/chalearn/dev/audio_16"
# train_text = "/data/seongjinpark/chalearn/train/gold_and_utts.tsv"
# test_text = "/data/seongjinpark/chalearn/dev/gold_and_utts.tsv"
audio_path = "/data/seongjinpark/MUStARD/utterances_final_16"
# train_text = "/work/seongjinpark/tomcat-speech/speech_recognition/data/chalearn/train.tsv"
# test_text = "/work/seongjinpark/tomcat-speech/speech_recognition/data/chalearn/dev.tsv"
train_text = "/work/seongjinpark/tomcat-speech/speech_recognition/data/mustard/train.tsv"
test_text = "/work/seongjinpark/tomcat-speech/speech_recognition/data/mustard/test.tsv"

train_csv = "/work/seongjinpark/tomcat-speech/speech_recognition/data/mustard/train.csv"
test_csv = "/work/seongjinpark/tomcat-speech/speech_recognition/data/mustard/dev.csv"

vocab_json = "/work/seongjinpark/tomcat-speech/speech_recognition/data/mustard/vocab.json"

train_dict = generate_dict(audio_path, train_text, train_csv, header=False)
test_dict = generate_dict(audio_path, test_text, test_csv, header=False)

chalearn_data = load_train_test(train_csv, test_csv)
print(chalearn_data)

Using custom data configuration default-da1e90b44f238bc4


Dataset csv downloaded and prepared to /work/seongjinpark/.cache/huggingface/datasets/csv/default-da1e90b44f238bc4/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.
DatasetDict({
    train: Dataset({
        features: ['file', 'text'],
        num_rows: 540
    })
    test: Dataset({
        features: ['file', 'text'],
        num_rows: 138
    })
})


In [5]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(chalearn_data["train"].remove_columns(["file"]))

,text
0,HEY MY FATHER'S HOUSE DOES THAT
1,YES THERE'S NOTHING QUITE LIKE THE SLIGHTLY WIDENED EYES OF MILDLY STARTLED
2,WELL THAT WADY TOOK HIGHWES PICTURES OF MY JUNK WAST NIGHT FOR CWAIGSWIST
3,CRAZY BITCH
4,ACTUALLY I YESTERDAY I WAS SMOKING AGAIN TODAY I AM SMOKING STILL
5,WEĀ€™RE NOT PEEKING
6,OH BOY I JUST CAN'T WATCH IT'S TOO SCARY
7,THE BLUNT INSTRUMENT THAT WILL BE THE FOCUS OF MY MURDER TRIAL
8,WELL DON'T 'CHA WANNA
9,AS YOU MAY KNOW I'VE BEEN EXPERIMENTING WITH ELEVATED ANXIETY LEVELS AND I THOUGHT WHAT BETTER WAY TO INCREASE MY DISCOMFORT THAN TO SUBJECT MYSELF TO AN EVENING OF TASTELESS UNCENSORED CROTCH TALK


In [6]:
import re

chars_to_ignore_regex = '[,\?\.\!\-\;\:\"<>€]'
int_put_space = '([\d])([A-z])'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    batch["text"] = batch["text"].replace("é", "e")
    batch["text"] = batch["text"].replace("ñ", "n")
    batch["text"] = batch["text"].replace("õ", "o")
    batch["text"] = batch["text"].replace("ā", "azz")
    batch["text"] = batch["text"].replace("™", "")

    return batch

chalearn_data = chalearn_data.map(remove_special_characters)

In [7]:
show_random_elements(chalearn_data["train"].remove_columns(["file"]))

,text
0,thank you
1,uh okay the fifth dentist caved and not they're all recommending trident
2,gee if only she were one and had no idea what a birthday was
3,certainly i'd like to raise two points number one i think they iarei talking about penises and number two these mimosas are kicking my little brown ass
4,yes i'd like a sevenday course of penicillin some uh syrup of ipecac to induce vomiting and a mint
5,see joe that's why your parents told you not to jump on your bed
6,all right cut let's pick again pick again
7,wow you look just like your son mrs tribbiani
8,thatos good just keep rubbing your head that will turn back time
9,chloe switch with me there's some guys here that got a crush on you


In [8]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = chalearn_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=chalearn_data.column_names["train"])

In [9]:
print(vocabs['train']['vocab'])

[['h', 'a', 'o', 'v', 'n', 'p', 'x', 'f', 'y', 'm', ' ', 'q', 'b', 's', 'u', 'j', '|', 'w', 'r', 'd', 'c', "'", 'e', 'i', 'l', 't', 'z', 'k', 'g']]


In [10]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'h': 0,
 'o': 1,
 'a': 2,
 'x': 3,
 'v': 4,
 'p': 5,
 'n': 6,
 'f': 7,
 'y': 8,
 'm': 9,
 ' ': 10,
 'q': 11,
 'b': 12,
 's': 13,
 'u': 14,
 'j': 15,
 '|': 16,
 'w': 17,
 'r': 18,
 'd': 19,
 'c': 20,
 "'": 21,
 'e': 22,
 'i': 23,
 'l': 24,
 't': 25,
 'z': 26,
 'k': 27,
 'g': 28}

In [11]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [12]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [13]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [14]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [15]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained("wav2vec2-base-chalearn")

In [16]:
import soundfile as sf

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

chalearn_data = chalearn_data.map(speech_file_to_array_fn, remove_columns=chalearn_data.column_names["train"], num_proc=4)

In [17]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

chalearn_prepared = chalearn_data.map(prepare_dataset, remove_columns=chalearn_data.column_names["train"], batch_size=8, num_proc=4, batched=True)

/work/seongjinpark/python3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
    

In [18]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [19]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [20]:
wer_metric = load_metric("wer")

In [21]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [22]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
model.freeze_feature_extractor()

In [24]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
  output_dir="wav2vec2-base-chalearn",
  group_by_length=True,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  evaluation_strategy="steps",
  num_train_epochs=40,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=chalearn_prepared["train"],
    eval_dataset=chalearn_prepared["test"],
    tokenizer=processor.feature_extractor,
)

In [26]:
trainer.train()

/work/seongjinpark/python3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


/work/seongjinpark/python3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/work/seongjinpark/python3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/work/seongjinpark/python3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/work/seongjinpark/python3/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dim

TrainOutput(global_step=2720, training_loss=2.4464933479533477, metrics={'train_runtime': 4277.5735, 'train_samples_per_second': 0.636, 'total_flos': 1.0836328725801907e+18, 'epoch': 40.0})

In [28]:
processor = Wav2Vec2Processor.from_pretrained("wav2vec2-base-chalearn/checkpoint-2500")
model = Wav2Vec2ForCTC.from_pretrained("wav2vec2-base-chalearn/checkpoint-2500")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [29]:
def map_to_result(batch):
    model.to("cuda")
    input_values = processor(
        batch["speech"],
        sampling_rate=batch["sampling_rate"],
        return_tensors="pt"
    ).input_values.to("cuda")

    with torch.no_grad():
        logits = model(input_values).logits
    
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_str"] = processor.batch_decode(pred_ids)[0]
    return batch

results = chalearn_data["test"].map(map_to_result)

In [30]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.849


In [31]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

,pred_str,target_text
0,<s>noh<s> i'<s> me<s>s<s>in<s>,amazing
1,<s>no<s> ro<s>se<s> [UNK]o<s>n to the c<s>o<s>rnor<s> and o<s>pen<s> hr s<s>t<s>a<s>,no rose go to the corner and open a stand
2,<s>h<s> you and<s>oh<s> s<s>ur<s>and wh<s>l w<s>er<s> <s>u<s>t <s>t whit i' we [UNK]et en<s>[UNK]<s>a<s>[UNK]ed to<s> <s>i<s>l<s> <s>a<s> w<s>e [UNK]et a itl a <s> h<s>o<s>use start a f<s>a<s>mil<s>y<s> <s> n jo<s>y our s<s>o<s>n<s>s<s>et<s> yea<s>r<s>s to<s>[UNK]ether<s>e<s>ve<s>al<s>d <s>youre yours<s>elf wo<s>m<s>an<s>,you and oh sure and while we're at it why don't we get engaged too why don't we get a little house start a family enjoy our sunset years together do you hear yourself woman
3,<s>yo're[UNK]onon<s> to te<s> me how you did<s> tha<s>t<s>,you're gonna have to tell me how you did that
4,<s> we [UNK]ot a bo<s>cke<s>,we got a box
5,n't ne<s>d a rea<s>s<s>on<s> it's m<s>y<s> h<s>ho<s>use<s>,i don't need a reason it's my house
6,<s> <s>oh<s> you'<s> ha<s>rdl<s>y outof the wo<s>d<s>s<s>the<s> you stil rube the ris<s> of i<s>f<s>e<s>ction<s> <s>a blu<s>d<s> c<s>lo<s>t<s> you'e po<s>s<s>ibility ha<s> an<s> <s>e<s>i<s>t<s>ent<s>if s<s>er<s>[UNK]<s>on<s> lhant a bua<s>rn<s> s<s>p<s>iter l<s>ake e<s>[UNK]s in your n<s>o<s>s<s>,oh you're hardly out of the woods no you still run the risk of infection a blood clot the possibility that an inattentive surgeon let a barn spider lay eggs in your nose
7,<s>e<s>yo <s>o<s> tha<s>t <s>a [UNK]lo<s>ry esti<s>f<s>in<s> wa<s>s<s> r<s>i[UNK]ht<s> <s>eventualy the ridhem <s>i<s>s<s> [UNK]o<s>in[UNK] to [UNK]et you<s>,yeah either that or gloria estefan was right eventually the rhythm is going to get you
8,<s>no<s> stupet pickd up thescro<s>p<s>ilo<s>s a<s>t in his <s>o<s>n<s>,no stuart picked out those throw pillows all on his own
9,<s>wh<s>y<s>caus she couldn<s> s<s>e<s>in[UNK] <s>a<s>nd<s> <s>pla<s>[UNK] a<s>t<s>a<s>r<s> <s>a<s>nd do<s> bo<s>th at the s<s>ame t<s>ime<s>,why because she can sing and play guitar and do both at the same time
